# Naive Bayes - SMS SPAM Detection

In [1]:
import pandas as pd

## Loading Dataset

In [2]:
# Source: https://archive.ics.uci.edu/dataset/228/sms+spam+collection

df = pd.read_csv("../data/sms_spam_collection.csv",
                 delimiter = "\t",
                 header = None,
                 names=["y", "sms"])
df.head()

,y,sms
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


In [3]:
df.shape

(5572, 2)

## Probability of Ham / Spam

In [4]:
df_general = df.groupby(['y']).count().reset_index()
df_general

,y,sms
0,ham,4825
1,spam,747


In [5]:
p_spam = 747 / 5572
p_ham = 4825 / 5572

print(p_spam, p_ham)

0.13406317300789664 0.8659368269921034


In [6]:
total_spam = df_general.loc[df_general["y"] == 'spam', 'sms'].values[0]
total_ham = df_general.loc[df_general["y"] == 'ham', 'sms'].values[0]

In [7]:
p_spam = total_spam / df.shape[0]
p_ham = total_ham / df.shape[0]

print(p_spam, p_ham)

0.13406317300789664 0.8659368269921034


## Text Preparation

### Converting to Lower Case

In [8]:
df = df.applymap(lambda x: x.lower() if isinstance(x, str) else x)
df.head()

<ipython-input-8-578a949386c2>:1: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df = df.applymap(lambda x: x.lower() if isinstance(x, str) else x)


,y,sms
0,ham,"go until jurong point, crazy.. available only ..."
1,ham,ok lar... joking wif u oni...
2,spam,free entry in 2 a wkly comp to win fa cup fina...
3,ham,u dun say so early hor... u c already then say...
4,ham,"nah i don't think he goes to usf, he lives aro..."


### Removing Special Characters

In [9]:
df["sms"] = df["sms"].str.replace("[^a-z 0-9]+", "", regex=True)
df.sample(10)

,y,sms
1452,ham,ya when are taking ure practical lessons i st...
4012,spam,please call our customer service representativ...
1551,ham,he says hi and to get your ass back to south t...
1603,ham,ok pa nothing problem
2354,spam,please call 08712402902 immediately as there i...
2542,ham,i dont can you send it to me plus hows mode
3500,ham,i hope you arnt pissed off but id would really...
2427,spam,valentines day special win over 1000 in our qu...
5454,ham,im just wondering what your doing right now
1983,ham,hey i will be late im at amk need to drink tea...


### Tokenizing, Stemming, Lemmatizing, Removing Stopwords

In [10]:
from nltk.tokenize import RegexpTokenizer
from nltk.stem import WordNetLemmatizer,PorterStemmer
from nltk.corpus import stopwords

In [11]:
stops = stopwords.words('english')
print(stops)

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

In [12]:
lemmatizer = WordNetLemmatizer()
stemmer = PorterStemmer()

def preprocess(sentence):
    tokenizer = RegexpTokenizer(r'\w+')
    tokens = tokenizer.tokenize(sentence)
    
    filtered_words = [w for w in tokens if len(w) > 2 if not w in stopwords.words('english')]
    stem_words=[stemmer.stem(w) for w in filtered_words]
    lemma_words=[lemmatizer.lemmatize(w) for w in stem_words]
    return " ".join(lemma_words)

df['sms'] = df['sms'].map(lambda s:preprocess(s)) 

In [13]:
df.sample(10)

,y,sms
1023,ham,still tonight
246,ham,late said websit didnt dont slipper
2562,ham,mayb pressi
3568,spam,collect valentin weekend pari inc flight hotel...
2775,ham,dude knw also telugutht gudk gud nyt
607,spam,xclusiveclubsaisai 2morow 285 soire special zo...
3621,ham,meant apolog text get drug ltgt night
595,ham,met stranger choos friend long world stand fri...
4082,ham,hurri home soup done
1293,ham,birthdat certif april real date today dont pub...


## Retrieving Unique Words

In [14]:
words = df['sms'].str.split().explode().drop_duplicates().values
words

array(['jurong', 'point', 'crazi', ..., 'now1', 'piti', 'soani'],
      dtype=object)

In [15]:
len(words)

7853

### Counting Words

In [16]:
df_count = df.set_index('y')['sms'].str.split().explode().reset_index().groupby(['sms', 'y']).size().to_frame()
df_count = df_count.reset_index()
df_count

,sms,y,0
0,008704050406,spam,2
1,0089mi,spam,1
2,0121,spam,1
3,01223585236,spam,1
4,01223585334,spam,2
...,...,...,...
8665,zoe,spam,1
8666,zogtoriu,ham,1
8667,zoom,ham,1
8668,zouk,spam,1


In [17]:
df_count.describe()

,0
count,8670.00000
mean,5.34579
std,16.90067
min,1.00000
25%,1.00000
50%,1.00000
75%,3.00000
max,366.00000


In [18]:
df_count[df_count[0] == 366]

,sms,y,0
1832,call,spam,366


### Calculating Probabilities

In [30]:
columns=['word', 'prob_ham', 'prob_spam']
df_p = pd.DataFrame(data=None, columns=columns)

for word in words:
    entry = []
    df_word = df_count[df_count["sms"] == word]
    if df_word.shape[0] > 0:
        count_word_spam = df_word.loc[df_word['y'] == 'spam', 0].values
        count_word_ham = df_word.loc[df_word['y'] == 'ham', 0].values
        
        if len(count_word_spam) > 0:
            count_word_spam = count_word_spam[0]
        else:
            count_word_spam = 1
            
        if len(count_word_ham) > 0:
            count_word_ham = count_word_ham[0]
        else:
            count_word_ham = 1
                    
        p_word_ham = count_word_ham / total_ham
        p_word_spam = count_word_spam / total_spam
        
        
        entry = [word, p_word_ham, p_word_spam]
        
        df_entry = pd.DataFrame(data=[entry], columns=columns)
        if df_p.shape[0] > 0:
            df_p = pd.concat([df_p, df_entry])
        else:
            df_p = df_entry

df_p = df_p.reset_index(drop=True)
df_p.sample(10)

,word,prob_ham,prob_spam
5547,friendshipmotherfatherteacherschildren,0.000207,0.001339
4345,rupaul,0.000207,0.001339
1309,bristol,0.000415,0.001339
2980,c52,0.000207,0.002677
1723,machiani,0.000207,0.001339
1051,meant,0.002902,0.001339
6985,7cfca1a,0.000207,0.001339
5598,thinl,0.000207,0.001339
5827,vatian,0.000207,0.001339
2840,ireneer,0.000207,0.001339


In [31]:
# these are the main components of the classifier that we need to store
# this is basically the model of naive bayes classifier

classifer_data = {"df": df_p, "p_ham": p_ham, "p_spam": p_spam}

In [32]:
df_p.describe()

,prob_ham,prob_spam
count,7852.000000,7852.000000
mean,0.000972,0.002822
std,0.003375,0.010043
min,0.000207,0.001339
25%,0.000207,0.001339
50%,0.000207,0.001339
75%,0.000415,0.001339
max,0.074611,0.489960


## Testing / Classification

In [21]:
df_sample = df.sample(1).values.tolist()
query = df_sample[0][1]
actual_class = df_sample[0][0]

print(df_sample)

query_words = query.split(' ')

[['ham', 'dont search love let love find that call fall love bcoz dont forc fall know smeon hold bslvyl']]


In [27]:
p_words_ham = 1
p_words_spam = 1

for word in query_words:
    df_p_word = df_p[df_p["word"] == word]

    p_words_ham *= df_p_word['prob_ham'].values[0]
    p_words_spam *= df_p_word['prob_spam'].values[0]

p_words = (p_words_ham * p_ham) + (p_words_spam * p_spam)

p_final_ham = p_words_ham * p_ham / p_words
p_final_spam = p_words_spam * p_spam / p_words

print(f"Ham: {p_final_ham:.5f}, Spam: {p_final_spam:.5f}")

if p_final_ham > p_final_spam:
    classified = 'ham'
else:
    classified = 'spam'

print("")
print(f"Predicted Class: {classified}")
print(f"Actual Class: {actual_class}")

Ham: 0.99997, Spam: 0.00003

Predicted Class: ham
Actual Class: ham


## Object Oriented Model

In [ ]:
class NaiveBayes():
    def __init__(self):
        pass
    def fit(self):
        pass
    def predict(self):
        pass